# Getting started with `Atoms`

This example will walk you through getting started with using `quippy` by playing with a small molecule.


First, make sure you have QUIP and `quippy` properly installed.  If you can run the cell below, then congratulations!  If not, then see the [Installation instructions](http://libatoms.github.io/QUIP/install.html).

In [ ]:
import numpy as np

import quippy
from quippy import Atoms

Remember, this is an interactive tutorial.  You are encouraged to run the cells yourself and even try editing the code to see what else you can do!

Now let's try getting our hands on an `Atoms` object; this object is required to do almost anything useful in `quippy`.

In [ ]:
struct = quippy.Atoms('methane.xyz')

OK, now what do we do with it?  Let's look at the Atoms source file and see what sort of information is encoded in it:

In [ ]:
!cat methane.xyz

We have positions, atomic symbols, atomic numbers, and a unit cell.  There's also some other information in there that we don't need right now.  But let's see how to access these properties from `quippy`:

In [ ]:
print("Positions:")
print(struct.get_positions())
print("Unit cell:")
print(struct.get_cell())
print("Atomic numbers:")
print(struct.get_atomic_numbers())

(Remember, units in quippy are always Ångstrøms and electronvolts: http://libatoms.github.io/QUIP/units.html)

You might want get an idea of what this structure actually looks like.  The native `xyz` format is supported by many open-source molecular viewers, like VMD (**TODO** link) and Avogadro (**TODO** link).  (**TODO** does `atomeye` work?  Try it here?)

**TODO**: Is there a simple in-notebook way we can view molecular structures?  Something with `matplotlib`, maybe?

Let's try another way to get information about the structure: This is a molecule, so we can check the bonds and angles.

In [ ]:
struct.get_distance(0, 1)

OK, that's a pretty reasonable C-H bond length in a methane molecule.  What about the other ones?

In [ ]:
[struct.get_distance(0, i) for i in range(1, 5)]

And how about an H-C-H angle?

In [ ]:
struct.get_angle([1, 0, 2]) * 180 / np.pi

which is the correct "tetrahedral angle" between the hydrogens in a methane molecule (it's equal to $\arccos\left(-\frac{1}{3}\right)$).

Note that the atom indices in these functions are *zero_based*, meaning the first atom (here, the carbon) is referred to by the index 0.

# ASE and `quippy`

If you've read some of the documentation, you may be aware of the following alternative method for getting the distance between the two atoms:

In [ ]:
struct.distance_min_image(0, 1)

DANGER WILL ROBINSON!  As you may have noticed, this isn't the correct bond distance.  In some circumstances the code may even just crash.  This is because the above function is derived from the underlying Fortran code, QUIP, rather than the `get_distance` function from before, which came from ASE.  The QUIP-derived functions can be very useful, but for now it'll just be too confusing to use them - let's stick with ASE.

So how do we tell whether a function is derived from QUIP or from ASE?  Well, let's take a look at this function's help message:

In [ ]:
help(struct.distance_min_image)

A bit confusing, yes, but take a look at the final line:

          Routine is wrapper around Fortran routine ``distance8_vec_atom`` defined in file :git:`src/libAtoms/Atoms_types.f95`.
          
It says this function is a _wrapper_ around something in the Fortran code, which means it's from QUIP and only works with _one-based_ indices (i.e. to this function, the first atom has the index 1).

In [ ]:
struct.distance_min_image(1, 2)

Whereas the ASE function has a different help string:

In [ ]:
help(struct.get_distance)

See that part, '`in module ase.atoms`'?  That means this function comes from ASE and works with zero-based indices.

You should _always_ do this check before using any `Atoms` function you're not yet familiar with.

## Useful QUIP equivalents

Just in case you ever want to work with the Fortran QUIP routines, here are some equivalents of the queries we've seen above:

In [ ]:
[struct.distance_min_image(1, i) for i in range(2, 6)]

In [ ]:
print(struct.cosine(1, 2, 3))
print(np.arccos(struct.cosine(1, 2, 3)) * 180 / np.pi)

Note that the _order_ of the atoms indices in `Atoms.cosine` is different as well: The central atom is the first one.

The properties can also be accessed as one-based arrays (called `FortranArray`s); these are also the transpose of the ASE arrays, so the row and column indices are swapped.

In [ ]:
struct.pos

In [ ]:
struct.lattice

In [ ]:
struct.Z

# Changing `Atoms`

There's a more complete tutorial on manipulating Atoms objects in ASE at [the ASE website](https://wiki.fysik.dtu.dk/ase/tutorials/manipulating_atoms.html); this is a shorter example just to get you started.

First, let's try reorienting our methane molecule in space.  The first hydrogen atom is already oriented along the Z axis; let's try to rotate the molecule so that the second one points along the X axis.

In [ ]:
r_h2 = struct.get_distance(0, 2, vector=True)
r_h2[2] = 0.0
np.arccos(r_h2[0] / np.linalg.norm(r_h2))

So we need to rotate it by about -0.9 radians about the Z axis.

But first, we want to keep the old methane molecule for comparison.  Let's copy it - note that this must be done _explicitly_; the Python assignment operator (`=`) only assigns a new _reference_ to the underlying object!  This can be one of the hardest things to understand for programmers coming from a different language, so make sure to read up on Python references and do some of their tutorials if you find this distinction confusing.

In [ ]:
struct_old = struct.copy()

In [ ]:
struct.rotate([0.0, 0.0, 1.0], -1.0 * np.arccos(r_h2[0] / np.linalg.norm(r_h2)))

In [ ]:
struct.get_positions()

Exercise: Now compare the positions of the rotated structure with those of the new structure; you may want to compute the RMS difference.

Now change the cell and make a supercell

# Generating structures from scratch

We could have also just used ASE functionality to pull a methane structure from its database:

In [ ]:
import ase
from ase.build import molecule

In [ ]:
me = molecule('CH4')

In [ ]:
me.get_positions()

Like the introductory tutorial at http://libatoms.github.io/QUIP/tutorial.html - use the `diamond` generator, view a supercell, do fun things with the structure...

# Computing the energy

Introduce `Potential`.  Use something like `IP SW` to get the energy of the silicon supercell.  Direct user to the list of potentials and the next tutorial.